In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import pandas_profiling
import seaborn as sns
import random as rn
import os
import scipy.stats as stats
import datetime
import calendar

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedKFold
from sklearn import metrics

from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

# 전처리

In [6]:
train = pd.read_csv('train.csv')

data_train_T=train['target']

target=[]
for i in range(len(data_train_T)):
    if data_train_T.iloc[i] < 250:
        target.append(1)
    elif data_train_T.iloc[i] < 1000:
        target.append(2)
    elif data_train_T.iloc[i] < 1800:
        target.append(3)
    else:
        target.append(4)
train["Range"] = target

train

,id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,NumDealsPurchases,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,target,Range
0,0,1974,Master,Together,46014.0,1,1,21-01-2013,21,10,...,7,0,0,0,0,0,0,0,541,2
1,1,1962,Graduation,Single,76624.0,0,1,24-05-2014,68,1,...,1,1,0,0,0,0,0,0,899,2
2,2,1951,Graduation,Married,75903.0,0,1,08-04-2013,50,2,...,3,0,0,0,0,0,0,0,901,2
3,3,1974,Basic,Married,18393.0,1,0,29-03-2014,2,2,...,8,0,0,0,0,0,0,0,50,1
4,4,1946,PhD,Together,64014.0,2,1,10-06-2014,56,7,...,7,0,0,0,1,0,0,0,444,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,1103,1956,Graduation,Together,46097.0,0,1,31-03-2013,11,5,...,4,0,0,0,0,0,0,0,241,1
1104,1104,1986,Graduation,Married,23477.0,1,0,21-10-2013,39,3,...,8,0,0,0,0,0,0,0,147,1
1105,1105,1975,Master,Married,37368.0,1,0,16-12-2013,4,1,...,6,1,0,0,0,0,0,0,30,1
1106,1106,1974,Graduation,Divorced,53034.0,1,1,30-05-2013,30,8,...,8,0,0,0,0,0,0,0,447,2


In [247]:
# 필요없는 feature drop

train = train.drop('id', axis = 1)
train = train.drop('Recency', axis = 1)

In [248]:
# 가장 과거 시점의 회사 등록일로부터 며칠이 지났는지를 뜻하는 Pass_Customer변수

train["Dt_Customer"] = pd.to_datetime(train["Dt_Customer"], format='%d-%m-%Y')
train_diff_date = train["Dt_Customer"] - train["Dt_Customer"].min()
train["Pass_Customer"] = [i.days for i in train_diff_date]
train = train.drop('Dt_Customer', axis = 1)

In [249]:
# 이상치 제거

# 100세 이상 제거
train = train.loc[(2022-train['Year_Birth']+1)<100, :].reset_index(drop=True)

In [250]:
# 나이 연령대로 나타내기

train['age'] = 2022 - train['Year_Birth'] + 1
train['age_group'] = train['age'] // 10 * 10
train = train.drop('age', axis = 1)
train = train.drop('Year_Birth', axis = 1)

In [251]:
# 총 구매횟수 낮을수록 카탈로그 비중 매우 낮음
# 총 구매횟수 높을수록 구매경로 별 비중의 차이 좁혀짐

train['TotalPurchases'] = train['NumCatalogPurchases']+train['NumStorePurchases']+train['NumWebPurchases']

train['RCatalogPurchases'] = np.where(train['TotalPurchases']==0, 0, train['NumCatalogPurchases'] / train['TotalPurchases'])
# train['RStorePurchases'] = np.where(train['TotalPurchases']==0, 0, train['NumStorePurchases'] / train['TotalPurchases'])
# train['RWebPurchases'] = np.where(train['TotalPurchases']==0, 0, train['NumWebPurchases'] / train['TotalPurchases'])

# train['dist'] = np.sqrt((train['RCatalogPurchases']-train['RStorePurchases'])**2 + (train['RStorePurchases']-train['RWebPurchases'])**2 + (train['RCatalogPurchases']-train['RWebPurchases'])**2)

In [252]:
# 캠페인 제안 수락 횟수

train['TotalCmp'] = train['AcceptedCmp1']+train['AcceptedCmp2']+train['AcceptedCmp3']+train['AcceptedCmp4']+train['AcceptedCmp5']+train['Response']

# train = train.drop('AcceptedCmp1', axis = 1)
train = train.drop('AcceptedCmp2', axis = 1)
train = train.drop('AcceptedCmp3', axis = 1)
train = train.drop('AcceptedCmp4', axis = 1)
# train = train.drop('AcceptedCmp5', axis = 1)

In [253]:
# 학력 Basic, 2ncycle 합치기

train['Education'] = np.where(train['Education'] == 'Basic', 'Under', train['Education'])
train['Education'] = np.where(train['Education'] == '2n Cycle', 'Under', train['Education'])

In [254]:
# 결혼 'Alone', 'YOLO', 'Absurd'을 'Single'에 합치기

train['Marital_Status'] = np.where(train['Marital_Status'] == 'Alone', 'Single', train['Marital_Status'])
train['Marital_Status'] = np.where(train['Marital_Status'] == 'YOLO', 'Single', train['Marital_Status'])
train['Marital_Status'] = np.where(train['Marital_Status'] == 'Absurd', 'Single', train['Marital_Status'])
#train['Marital_Status'] = np.where(train['Marital_Status'] == 'Married', 'Together', train['Marital_Status'])


In [255]:
# 자녀 수, 청소년 수 합쳐서 Binary 변환

train['Dependents'] = train['Kidhome'] + train['Teenhome']
train['Dependents'] = np.where(train['Dependents'] >= 1, 1, train['Dependents'])

In [256]:
# 높은 skewness를 가지고 있는 NumDealsPurchases 변수
# Log transformation
# log_trans = train['NumDealsPurchases'].map(lambda i: np.log(i) if i > 0 else 0)

# Yeo-Johnson transformation
jy = PowerTransformer(method = 'yeo-johnson')
jy.fit(train['NumDealsPurchases'].values.reshape(-1, 1))
x_yj = jy.transform(train['NumDealsPurchases'].values.reshape(-1, 1))

train['NumDealsPurchases'] = x_yj

In [257]:
#  Education와 Marital_Status 피쳐 Label Encoding

def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype == 'object':
            label_map = {'unknown' : 0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i
            label_maps[col] = label_map
    return label_maps

def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype == 'object':
            dataframe[col] = dataframe[col].map(label_map[col])
    return dataframe

train = label_encoder(train, make_label_map(train))

In [258]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Education          1106 non-null   int64  
 1   Marital_Status     1106 non-null   int64  
 2   Income             1106 non-null   float64
 3   Kidhome            1106 non-null   int64  
 4   NumDealsPurchases  1106 non-null   float64
 5   NumWebPurchases    1106 non-null   int64  
 6   NumStorePurchases  1106 non-null   int64  
 7   NumWebVisitsMonth  1106 non-null   int64  
 8   AcceptedCmp5       1106 non-null   int64  
 9   AcceptedCmp1       1106 non-null   int64  
 10  Complain           1106 non-null   int64  
 11  Response           1106 non-null   int64  
 12  target             1106 non-null   int64  
 13  Range              1106 non-null   int64  
 14  Pass_Customer      1106 non-null   int64  
 15  age_group          1106 non-null   int64  
 16  TotalPurchases     1106 

In [259]:
# 테스트 데이터 전처리

test = pd.read_csv('test.csv')

test = test.drop('id', axis = 1)
test = test.drop('Recency', axis = 1)
# test = test.drop('NumDealsPurchases', axis = 1)

test["Dt_Customer"] = pd.to_datetime(test["Dt_Customer"], format='%d-%m-%Y')
train_diff_date = test["Dt_Customer"] - test["Dt_Customer"].min()
test["Pass_Customer"] = [i.days for i in train_diff_date]
test = test.drop('Dt_Customer', axis = 1)

test['age'] = 2022 - test['Year_Birth'] + 1
test['age_group'] = test['age'] // 10 * 10
test = test.drop('age', axis = 1)
test = test.drop('Year_Birth', axis = 1)

test['TotalPurchases'] = test['NumCatalogPurchases']+test['NumStorePurchases']+test['NumWebPurchases']

test['RCatalogPurchases'] = np.where(test['TotalPurchases']==0, 0, test['NumCatalogPurchases'] / test['TotalPurchases'])

test['TotalCmp'] = test['AcceptedCmp1']+test['AcceptedCmp2']+test['AcceptedCmp3']+test['AcceptedCmp4']+test['AcceptedCmp5']+test['Response']

test = test.drop('AcceptedCmp2', axis = 1)
test = test.drop('AcceptedCmp3', axis = 1)
test = test.drop('AcceptedCmp4', axis = 1)

test['Education'] = np.where(test['Education'] == 'Basic', 'Under', test['Education'])
test['Education'] = np.where(test['Education'] == '2n Cycle', 'Under', test['Education'])

test['Marital_Status'] = np.where(test['Marital_Status'] == 'Alone', 'Single', test['Marital_Status'])
test['Marital_Status'] = np.where(test['Marital_Status'] == 'YOLO', 'Single', test['Marital_Status'])
test['Marital_Status'] = np.where(test['Marital_Status'] == 'Absurd', 'Single', test['Marital_Status'])

test['Dependents'] = test['Kidhome'] + test['Teenhome']
test['Dependents'] = np.where(test['Dependents'] >= 1, 1, test['Dependents'])

jy = PowerTransformer(method = 'yeo-johnson')
jy.fit(test['NumDealsPurchases'].values.reshape(-1, 1))
x_yj = jy.transform(test['NumDealsPurchases'].values.reshape(-1, 1))

test['NumDealsPurchases'] = x_yj


test = label_encoder(test, make_label_map(test))

# 학습

In [260]:
import numpy as np
from sklearn.metrics import mean_squared_error

def NMAE(true, pred):

    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    
    return score

In [261]:
data_train_X = train.drop('target', axis = 1)
data_train_y = train['target']

In [268]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True) #총 10번의 fold 진행
n = 0 #x번째 fold인지 기록

fold_target_pred = []
fold_score = []

for train_index, valid_index in skf.split(data_train_X, data_train_X['Range']): #range 기준으로 stratified k fold 진행
    n += 1
    
    val_pred_name = [] #validation pred model 이름 저장
    val_pred = []      #validation set pred 결과 저장
    target_pred = []   #test set pred 결과 저장
    
    train_X = np.array(data_train_X.drop("Range", axis = 1)) #분배된 학습을 위해 생성한 Range feature 제거
    train_Y = np.array(data_train_y)
    
    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_Y[train_index], train_Y[valid_index]
    
    X_test = np.array(test)

    ### Create Model ###
    
    ###모델을 생성하고 집어넣으면 됩니다.
    
    ### LGBMRegressor ###
#     model = LGBMRegressor(n_estimators=134,max_depth=16,random_state = 42) #추가적으로 하이퍼파라미터 튜닝 필요
#     model.fit(X_train, y_train) # 모델 학습
    
#     val_pred_name.append("LGBMRegressor")      # 모델 이름 저장
#     val_pred.append(model.predict(X_valid))   # validation set pred 결과 저장
#     target_pred.append(model.predict(X_test)) # test set pred 결과 저장
    
    ### XGBRegressor ###
    model = XGBRegressor(n_estimators=194, max_depth=7, random_state = 42) #추가적으로 하이퍼파라미터 튜닝 필요
    model.fit(X_train, y_train)
    
    val_pred_name.append("XGBRegressor")      # 모델 이름 저장
    val_pred.append(model.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model.predict(X_test)) # test set pred 결과 저장
    
    ### CatBoostRegressor ###
    model = CatBoostRegressor(n_estimators=1200,max_depth=8,random_state = 42, silent=True) #추가적으로 하이퍼파라미터 튜닝 필요
    model.fit(X_train, y_train)
    
    val_pred_name.append("CatBoostRegressor")      # 모델 이름 저장
    val_pred.append(model.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model.predict(X_test)) # test set pred 결과 저장

    ### RandomForestRegressor ###
    model = RandomForestRegressor(n_estimators=177,max_depth=16,random_state = 42, criterion="mae") #추가적으로 하이퍼파라미터 튜닝 필요
    model.fit(X_train, y_train)
    
    val_pred_name.append("RandomForestRegressor")      # 모델 이름 저장
    val_pred.append(model.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model.predict(X_test)) # test set pred 결과 저장    
    
    ### voting ###
    
    ### average validation pred ###
    preds = np.array(val_pred[0])
    for i in range(1, len(val_pred)):
        preds += val_pred[i]
    
    preds = preds/len(val_pred)
    
    ### average target pred ###
    target_preds = np.array(target_pred[0])
    for i in range(1, len(target_pred)):
        target_preds += target_pred[i]
    
    target_preds = target_preds/len(target_pred)
    
    fold_target_pred.append(target_preds) # append final target pred
    
    print("========== fold %d ==========" %(n))
    for i in range(len(val_pred)):
        print("%s model NMAE : %0.4f" %(val_pred_name[i], NMAE(y_valid, val_pred[i].astype(int))))
        
    print("==============================")
    print("Average NMAE %0.4f" %(NMAE(y_valid, preds.astype(int))))
    print("")
    
    fold_score.append(NMAE(y_valid, preds.astype(int)))

total_score = fold_score[0]
for i in range(2, len(fold_score), 1):
    total_score += fold_score[i]
total_score = total_score/(len(fold_score))    

print("==============================")
print("Total Average NMAE %0.4f" %(total_score)) #최종 average score 출력

========== fold 1 ==========
XGBRegressor model NMAE : 0.1969
CatBoostRegressor model NMAE : 0.2075
RandomForestRegressor model NMAE : 0.2262
Average NMAE 0.2052

========== fold 2 ==========
XGBRegressor model NMAE : 0.2015
CatBoostRegressor model NMAE : 0.1676
RandomForestRegressor model NMAE : 0.1863
Average NMAE 0.1756

========== fold 3 ==========
XGBRegressor model NMAE : 0.1384
CatBoostRegressor model NMAE : 0.1515
RandomForestRegressor model NMAE : 0.1447
Average NMAE 0.1336

========== fold 4 ==========
XGBRegressor model NMAE : 0.1940
CatBoostRegressor model NMAE : 0.1794
RandomForestRegressor model NMAE : 0.1963
Average NMAE 0.1818

========== fold 5 ==========
XGBRegressor model NMAE : 0.2057
CatBoostRegressor model NMAE : 0.2102
RandomForestRegressor model NMAE : 0.2247
Average NMAE 0.2039

========== fold 6 ==========
XGBRegressor model NMAE : 0.1859
CatBoostRegressor model NMAE : 0.1751
RandomForestRegressor model NMAE : 0.2117
Average NMAE 0.1782

========== fold 7 ====

# 제출

In [269]:
### average target pred ###
final_pred = np.array(fold_target_pred[0])
for i in range(1, len(fold_target_pred)):
    final_pred += fold_target_pred[i]

final_pred = final_pred/len(fold_target_pred)
final_pred = final_pred.astype(int)

submission = pd.read_csv("./sample_submission.csv")

submission['target'] = final_pred
submission.head()

submission.to_csv("Submit_7.csv",index=False)